In [1]:
# visualize feature maps output from each block in the vgg model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import Model
from matplotlib import pyplot
from numpy import expand_dims
import numpy as np
from itertools import chain
from keras import backend as K
from keras.layers.convolutional import MaxPooling3D
from vvvv import VVVVClient
import keras
import time

Using TensorFlow backend.


In [4]:
# load the model
model = VGG16()

# redefine model to output right after the first hidden layer
ixs = [2, 5, 9, 13, 17]
outputs = [model.layers[i].output for i in ixs]
model = Model(inputs=model.inputs, outputs=outputs)

# load the image with the required shape
img = load_img(r"C:\Users\adchu\Desktop\0.png", target_size=(224, 224))

# convert the image to an array
img = img_to_array(img)

# expand dimensions so that it represents a single 'sample'
img = expand_dims(img, axis=0)
# prepare the image (e.g. scale pixel values for the vgg)
img = preprocess_input(img)

# get feature map for first hidden layer
feature_maps = model.predict(img)
                                        
toMax = []
                                        
for i in range(len(feature_maps)):
    toMax.append(feature_maps[i][0].mean(2).flatten().tolist())


toMaxArray = []
#print(toMaxArray)
cycle = 0
print(cycle)

delay = 0.5

func = [3, 2, 1, 1, 1]

for i in range (len(toMax)):
    n = feature_maps[i][0].shape[0]
    pool = keras.layers.MaxPooling2D(pool_size=(func[i], func[i]))
    b = K.constant(feature_maps[i][0].mean(2).reshape(-1, n, n, 1))
    out = K.eval(pool(b))
    out = (out - out.min()) / (out.max() - out.min())
    #print(out.shape)
    toMaxArray += [out.flatten().tolist()]

vc = VVVVClient('127.0.0.1', int('12000'))
for i in range(len(toMaxArray)):
    vc.send_msg('/conv{}'.format(i), toMaxArray[i])
    vc.send_msg('/trig0', str(1))
    vc.send_msg('/trig0', str(0))
time.sleep(delay)
    

while True:
    cycle += 1
    # load the image with the required shape
    img = load_img(r"C:\Users\adchu\Desktop\1.png", target_size=(224, 224))

    # convert the image to an array
    img = img_to_array(img)

    # expand dimensions so that it represents a single 'sample'
    img = expand_dims(img, axis=0)
    # prepare the image (e.g. scale pixel values for the vgg)
    img = preprocess_input(img)

    # get feature map for first hidden layer
    feature_maps = model.predict(img)


    toMax = []

    for i in range(len(feature_maps)):
        toMax.append(feature_maps[i][0].mean(2).flatten().tolist())

    #print(toMax[4])

    toMaxArray = []
    #print(toMaxArray)
    print(cycle)
    
    func = [3, 2, 1, 1, 1]

    for i in range (len(toMax)):
        n = feature_maps[i][0].shape[0]
        pool = keras.layers.MaxPooling2D(pool_size=(func[i], func[i]))
        b = K.constant(feature_maps[i][0].mean(2).reshape(-1, n, n, 1))
        out = K.eval(pool(b))
        out = (out - out.min()) / (out.max() - out.min())
        #print(out.shape)
        toMaxArray += [out.flatten().tolist()]

    vc = VVVVClient('127.0.0.1', int('12000'))
    for i in range(len(toMaxArray)):
        vc.send_msg('/conv{}'.format(i), toMaxArray[i])
        vc.send_msg('/trig1', str(1))
        vc.send_msg('/trig1', str(0))
    time.sleep(delay)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 